Please replace the **YYYYYY** with the **absolute path** of the install directory of OpenPose, and **XXXXXX** with the **absolute path** of your video

In [0]:
!YYYYYY/openpose/build/examples/openpose/openpose.bin --video XXXXXX --write_json ./test/op_json --render_pose 0 --number_people_max 1 --keypoint_scale 3 --display 0 --face

If you have installed tensorflow 1.X and Keras, **comment this block below**

In [0]:
!pip install tensorflow==1.15.0
!pip install Keras

In [0]:
!pip install opencv-python
!pip install matplotlib

Read input video (Change **FILEPATH** to the video you are going to use)

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import csv
import cv2

Input your own json folder path and your label csv file path here to './drive/My Drive/636/test/'

In [0]:
N_FACE = 70
N_POSE = 8
N_FEATURES = (N_FACE + N_POSE) * 2
MODEL_FOLDER = './model/'
JSON_FOLDER = './test/op_json/'
FOLDER = './'
NAME = '827008769'

COL_NAMES = ['frame']
COL_NAMES += ['f_' + str(i) for i in range(N_FACE * 2)]
COL_NAMES += ['p_' + str(i) for i in range(N_POSE * 2)]
droplist_f = set([(i * 3 + 2) for i in range(N_FACE)])
droplist_p = set([(i * 3 + 2) for i in range(N_POSE)])
idx_f = [i for i in range(N_FACE * 3) if (i not in droplist_f)]
idx_p = [i for i in range(N_POSE * 3) if (i not in droplist_p)]

In [0]:
# collect the json output of openpose and combine them into csv
def json_to_csv(folderpath, csv_path):
  idx = 0
  write_file = open(csv_path, 'w')
  csv_writer = csv.writer(write_file)
  csv_writer.writerow(COL_NAMES)
  for name in sorted(os.listdir(folderpath)): 
    if name[-4:] == 'json':
      filepath = os.path.join(folderpath, name)
      if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
          data = json.load(f)['people']
          if len(data) > 0:
            row = [name]
            row_face = [float(data[0]['face_keypoints_2d'][j]) for j in idx_f]
            row_pose = [float(data[0]['pose_keypoints_2d'][j]) for j in idx_p]
            row += row_face + row_pose
            row = np.array(row)
            csv_writer.writerow(row)
      idx += 1

def get_csv(name):
  csv_path = FOLDER + name + '.csv'
  json_to_csv(JSON_FOLDER, csv_path)

Reshape Data into Segments and Prepare for Keras

In [0]:
# slice every TIME_STEP frame as a segment until from 1st frame to last, if null, make up with 0
def get_slices(df, TIME_STEP, fps):
  slices = []
  ts = []
  df = df[df.columns[1:]]
  for i in range(df.shape[0]):
    ts.append(i/fps)
    if i + 1 < TIME_STEP:
      slice_ = df.values[df.index[0 : i + 1]]
      fillzero = np.zeros((TIME_STEP - i - 1, N_FEATURES))
      slice_ = np.concatenate((slice_, fillzero), axis=0)
      slices.append(slice_)
    else:
      slices.append(df.values[df.index[i + 1 - TIME_STEP : i + 1]])

  slices_arr = np.asarray(slices, dtype= np.float32)
  ts_arr = np.asarray(ts, dtype= np.float32)
  return slices_arr, ts_arr

Load model and weight

In [0]:
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, LSTM, Dropout, Flatten, Conv1D, Activation, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import History, EarlyStopping, ModelCheckpoint
from keras.models import load_model

TensorFlow 1.x selected.


Using TensorFlow backend.


Draw figure

In [0]:
import matplotlib.pyplot as plt

def drawplt(time, pred, name):
  plt.figure(figsize=(10,8))
  plt.plot(time, pred, 'b', label='prediction', markersize=3, color='red')
  plt.title('Prediction over time')
  my_y_ticks = np.arange(0, 1.1, 0.1)
  plt.yticks(my_y_ticks)
  plt.legend()
  plt.savefig(name)

Generate json

In [0]:
def to_json(name, ts_arr, y_pred):
  np_pred = np.array((ts_arr, y_pred.flatten())).T
  pred_list = np_pred.tolist()
  pred_dict = {"cry":pred_list}
  pred_json = json.dumps(pred_dict)
  with open(name, 'w') as json_file:
    json_file.write(pred_json)

Get fps

In [0]:
videoCapture = cv2.VideoCapture("video.mp4")
fps = videoCapture.get(cv2.CAP_PROP_FPS)

Json to csv

In [0]:
get_csv(NAME)

In [0]:
def get_pred(X, model, idx):
  # get prediction
  y_pred = model.predict(X)
  # draw plot
  fig_name = FOLDER + NAME + '_Part' + str(idx) + '.png'
  drawplt(ts_arr, y_pred, fig_name)
  # output json file
  js_name = FOLDER + NAME + '_Part' + str(idx) + '.json'
  to_json(js_name, ts_arr, y_pred)

**Automation_Part8**

In [0]:
X = pd.read_csv(FOLDER + NAME + '.csv')
# load model
model = load_model(MODEL_FOLDER + 'part8/big_7859.h5')
# get input and timestep
X_arr, ts_arr = get_slices(X, 10, fps)
# reshape input
X_arr4 = X_arr.reshape(X_arr.shape[0], 40, 39, 1)
get_pred(X_arr4, model, 8)

**Automation_Part6**

In [0]:
# load model
model = load_model(MODEL_FOLDER + 'model4_63.h5')
get_pred(X_arr, model, 6)

**Automation_Part5**

In [0]:
# load model
model = load_model(MODEL_FOLDER + 'bi_rmsprop.h5')
get_pred(X_arr, model, 5)

PART4

In [0]:
# load model
model = load_model(MODEL_FOLDER + 'origin_rmsprop.h5')
get_pred(X_arr, model, 4)